In [ ]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import plotly.express as px

if 'src' not in os.listdir('.'):
    os.chdir('../')

import src.utils.models as M
import src.utils.constants as C
import src.utils.evaluation as E
from src.utils.runner import Runner
from config.debugger_config import debugger_experiments
from config.aggregate_config import aggregate_experiments
from config.increment_config import increment_experiments
from scripts.run_experiment import run_experiment
from src.utils.dataframe import build_metadata_df, build_df

In [ ]:
# runner = Runner(parallel=False)
runner = Runner(parallel=True)

runner.run(run_experiment, [
    *debugger_experiments,
    *aggregate_experiments,
    *increment_experiments,
])

# run_experiment(debugger_experiments[0])

In [ ]:
metadata_df = build_metadata_df('results/**/config.json')
# metadata_df = metadata_df[metadata_df[C.EXPERIMENT_NAME].str.contains('Debugger|Aggregate|Increment')]
metadata_df

In [ ]:
experiments_df = build_df(metadata_df, C.EXPERIMENT_LOGGER)
experiments_df.sort_values(by=C.OBSERVATION_TIME, inplace=True)
experiments_df

In [ ]:
px.scatter(
    experiments_df,
    x=C.ITERATION,
    y=C.Y_PRED,
    color=C.EXPERIMENT_NAME,
    title='Predictions over time',
    hover_data=experiments_df.columns
).show()

In [ ]:
px.scatter(
    experiments_df,
    x=C.ITERATION,
    y=C.MAE,
    color=C.EXPERIMENT_NAME,
    title='MAE over time',
    hover_data=experiments_df.columns
)

In [ ]:
px.scatter(
    experiments_df,
    x=C.OBSERVATION_TIME,
    y=C.MAE,
    color=C.EXPERIMENT_NAME,
    title='MAE on observation time',
    hover_data=experiments_df.columns
)

In [ ]:
px.box(
    experiments_df,
    y=C.MAE,
    title=C.MAE,
    labels={'y': C.MAE},
    color=C.EXPERIMENT_NAME,
    hover_data=experiments_df.columns
).update_traces(boxmean=True).show()

In [ ]:
px.scatter(
    experiments_df,
    x=C.ITERATION,
    y=C.HITS,
    color=C.EXPERIMENT_NAME,
    title='Hits over time',
    hover_data=experiments_df.columns
)

In [ ]:
experiments_eval_df = E.evaluate_experiment_group(experiments_df)
experiments_eval_df.sort_values(by=C.MAE)

In [ ]:
px.bar(
    experiments_eval_df.sort_values(by=C.MAE).melt(
        id_vars=C.EXPERIMENT_NAME,
        value_vars=[C.RMSE, C.MAE]
    ),
    x=C.EXPERIMENT_NAME,
    y='value',
    color='variable',
    barmode='group',
    labels={'value': 'Loss Metrics', 'variable': 'Metric'},
    title='Loss Metrics (RMSE and MAE)',
)

In [ ]:
px.bar(
    experiments_eval_df.sort_values(by=C.MBE),
    x=C.EXPERIMENT_NAME,
    y=C.MBE,
    title='Loss Metrics (MBE)'
)

In [ ]:
cachais_df = build_df(metadata_df, C.CACHAI_LOGGER)
cachais_df

In [ ]:
cachais_eval_df = E.evaluate_cachai_group(cachais_df)
cachais_eval_df

In [ ]:
px.bar(
    cachais_eval_df.sort_values(by=C.CACHE_HIT_PRECISION).melt(
        id_vars=C.EXPERIMENT_NAME,
        value_vars=[
            C.CACHE_SERVE_RATE, C.CACHE_HIT_PRECISION, C.CACHE_HIT_ACCURACY, C.CACHE_STALE_RATE, C.CACHE_MISS_RATE
        ]
    ),
    x=C.EXPERIMENT_NAME,
    y='value',
    color='variable',
    barmode='group',
    labels={'value': 'Rate', 'variable': 'Metric'},
    title='Cache Metrics'
)

In [ ]:
px.bar(
    cachais_eval_df.sort_values(by=C.CACHE_HIT_TOTAL),
    x=C.EXPERIMENT_NAME,
    y=C.CACHE_HIT_TOTAL,
    title='Cache Hit Total'
)